In [5]:
import pandas as pd
import pymysql
import logging

# Logging setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def test_db_connection():
    """Test database connection and run queries"""
    try:
        # Connect to Genboost database
        conn = pymysql.connect(
            host='10.0.0.46',
            user='Merchflow',
            password='Gen_Merch2024!+',
            database='Genboost',
            charset='utf8mb4'
        )
        
        logger.info("Database connection successful!")
        
        # Test 1: Data size check for 24 months
        logger.info("Testing data size for 24 months...")
        size_query = """
        SELECT 
            COUNT(*) as total_rows,
            MIN(`date`) as min_date,
            MAX(`date`) as max_date,
            COUNT(DISTINCT `date`) as unique_dates
        FROM Genboost.history_sales 
        WHERE `date` >= DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 24 MONTH), '%Y-%m-01')
        """
        
        size_df = pd.read_sql(size_query, conn)
        logger.info(f"Data size info: {size_df.to_dict('records')[0]}")
        
        # Test 2: Sample aggregated data
        logger.info("Testing sample aggregated data...")
        sample_query = """
        SELECT 
            date, store_code, product_id,
            SUM(discount_amount) as discount_amount,
            SUM(net_amount_wovat) as net_amount_wovat,
            SUM(net_quantity) as net_quantity,
            SUM(net_amount_wovat) / SUM(net_quantity) as unit_price
        FROM Genboost.history_sales
        WHERE `date` >= DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 24 MONTH), '%Y-%m-01')
        GROUP BY date, store_code, product_id
        LIMIT 10
        """
        
        sample_df = pd.read_sql(sample_query, conn)
        logger.info(f"Sample data shape: {sample_df.shape}")
        logger.info(f"Sample data:\n{sample_df.head()}")
        
        # Test 3: Other tables
        logger.info("Testing other tables...")
        
        product_query = "SELECT COUNT(*) as product_count FROM Genboost.dim_product"
        product_count = pd.read_sql(product_query, conn)
        logger.info(f"Product count: {product_count.iloc[0]['product_count']}")
        
        cluster_query = "SELECT COUNT(*) as cluster_count FROM Genboost.store_clustering"
        cluster_count = pd.read_sql(cluster_query, conn)
        logger.info(f"Store cluster count: {cluster_count.iloc[0]['cluster_count']}")
        
        calendar_query = "SELECT COUNT(*) as calendar_count FROM Genboost.dim_calendar"
        calendar_count = pd.read_sql(calendar_query, conn)
        logger.info(f"Calendar count: {calendar_count.iloc[0]['calendar_count']}")
        
        conn.close()
        logger.info("All tests completed successfully!")
        
    except Exception as e:
        logger.error(f"Error: {e}")
        raise

if __name__ == "__main__":
    test_db_connection()


INFO:__main__:Database connection successful!
INFO:__main__:Testing data size for 24 months...
C:\Users\FromHell\AppData\Local\Temp\ipykernel_27956\139884243.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  size_df = pd.read_sql(size_query, conn)
INFO:__main__:Data size info: {'total_rows': 3941493, 'min_date': Timestamp('2023-08-01 00:00:00'), 'max_date': Timestamp('2025-08-24 00:00:00'), 'unique_dates': 755}
INFO:__main__:Testing sample aggregated data...
C:\Users\FromHell\AppData\Local\Temp\ipykernel_27956\139884243.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sample_df = pd.read_sql(sample_query, conn)
INFO:__main__:Sample data shape: (10, 7)
INFO:__main__:Sampl

In [13]:
import pandas as pd
import pymysql
import traceback

def run_query(query: str) -> pd.DataFrame | None:
    conn = None
    try:
        conn = pymysql.connect(
            host='10.0.0.46',
            user='Merchflow',
            password='Gen_Merch2024!+',
            database='Genboost',
            charset='utf8mb4'
        )
        df = pd.read_sql(query, conn)
        # Örnek çıktı
        print(f"Toplam satır: {len(df)}")
        print(df.head(10).to_string(index=False))
        return df
    except Exception as e:
        print("Hata:", e)
        traceback.print_exc()
        return None
    finally:
        if conn:
            conn.close()

query = """
        SELECT 
            date, store_code, product_id,
            SUM(discount_amount) as discount_amount,
            SUM(net_amount_wovat) as net_amount_wovat,
            SUM(net_quantity) as net_quantity,
            SUM(net_amount_wovat) / SUM(net_quantity) as unit_price
        FROM Genboost.history_sales
        WHERE `date` >= DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 24 MONTH), '%Y-%m-01')
        GROUP BY date, store_code, product_id
"""

df = run_query(query)

# İstersen burada da kontrol edebilirsin:
if df is not None:
    print("\nİlk 5 satır tekrar:")
    print(df.head().to_string(index=False))


C:\Users\FromHell\AppData\Local\Temp\ipykernel_20100\2842824848.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Toplam satır: 1476481
      date store_code                product_id  discount_amount  net_amount_wovat  net_quantity  unit_price
2023-08-01       0101          101295860BEYAZ43             0.00           1509.08           2.0      754.54
2023-08-01       0101          101330069BEYAZ40            45.30            956.34           2.0      478.17
2023-08-01       0101          101330211BEYAZ39             0.00           1636.34           2.0      818.17
2023-08-01       0101          101342567BEYAZ42            28.24           1327.26           2.0      663.63
2023-08-01       0101           101369449LACI43             0.00            563.62           2.0      281.81
2023-08-01       0101           101373117HAKI41             0.00            599.98           2.0      299.99
2023-08-01       0101      11016-6ENPepper42-43             0.00           1834.54           2.0      917.27
2023-08-01       0101        12228020-WHTWhiteL             0.00           1272.70           2.0      636.

In [16]:
import pandas as pd
import pymysql, csv, re

def _detect_csv_format(path, encoding="utf-8"):
    # İlk birkaç KB ile ayraç ve başlık var mı tespit et
    with open(path, "r", encoding=encoding, newline="") as f:
        sample = f.read(4096)
        try:
            sniffer = csv.Sniffer()
            dialect = sniffer.sniff(sample, delimiters=[",",";","\t","|"])
            has_header = sniffer.has_header(sample)
            delimiter = dialect.delimiter
        except Exception:
            # fallback: ; sık görülür
            delimiter, has_header = ";", True
    # Satır sonu tahmini
    if "\r\n" in sample:
        lineterm = r"\r\n"
    else:
        lineterm = r"\n"
    return delimiter, has_header, lineterm

def _sanitize_mysql_ident(name: str, maxlen=60):
    name = name.strip().replace("\ufeff","")     # BOM temizle
    name = re.sub(r"\s+", "_", name)             # boşluk -> _
    name = re.sub(r"[^\w$]", "_", name)          # harf/rakam/_ dışı -> _
    if name and name[0].isdigit():
        name = "_" + name
    return name[:maxlen] or "_col"

def upload_csv_fast(csv_path, table_name, encoding="utf-8"):
    # 0) CSV biçimini algıla
    delim, has_header, lineterm = _detect_csv_format(csv_path, encoding=encoding)

    # 1) Başlıkları doğru ayraçla oku
    df_head = pd.read_csv(csv_path, nrows=0, sep=delim, engine="python", encoding=encoding)
    raw_cols = df_head.columns.tolist()

    # 2) Kolon adlarını güvenli hale getir + çakışma çöz
    safe_cols, used = [], set()
    for c in raw_cols:
        s = _sanitize_mysql_ident(c)
        base, k = s, 1
        while s in used:
            suff = f"_{k}"
            s = (base[: max(1, 60 - len(suff))] + suff)
            k += 1
        used.add(s)
        safe_cols.append(s)

    col_defs = ",\n  ".join(f"`{c}` TEXT" for c in safe_cols)  # hızlı başlangıç: TEXT
    col_list = ",".join(f"`{c}`" for c in safe_cols)

    # 3) MySQL'e bağlan
    conn = pymysql.connect(
        host='10.0.0.46',
        user='Merchflow',
        password='Gen_Merch2024!+',
        database='Genboost',
        charset='utf8mb4',
        local_infile=True
    )
    try:
        with conn.cursor() as cur:
            cur.execute("SET SESSION local_infile=1;")
            # 4) Tabloyu oluştur (yoksa)
            create_sql = f"""
            CREATE TABLE IF NOT EXISTS `{table_name}` (
              {col_defs}
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
            """
            cur.execute(create_sql)

            # 5) LOAD DATA — ayraç/satır sonu dinamik
            mysql_delim = {"\t": r"\t"}.get(delim, delim)  # \t kaçış
            load_sql = f"""
            LOAD DATA LOCAL INFILE %s
            INTO TABLE `{table_name}`
            CHARACTER SET utf8mb4
            FIELDS TERMINATED BY '{mysql_delim}'
            OPTIONALLY ENCLOSED BY '"'
            ESCAPED BY '\\\\'
            LINES TERMINATED BY '{lineterm}'
            {"IGNORE 1 LINES" if has_header else ""}
            ({col_list});
            """
            cur.execute(load_sql, (csv_path,))
        conn.commit()
        print(f"Yüklendi → Genboost.{table_name}  (kolon sayısı: {len(safe_cols)}, ayraç: '{delim}')")
    finally:
        conn.close()


In [18]:
CSV_PATH = "replenishment_results.csv"
TABLE_NAME = "replenishment_results"
upload_csv_fast(CSV_PATH, TABLE_NAME)


OperationalError: (1229, "Variable 'local_infile' is a GLOBAL variable and should be set with SET GLOBAL")